# Getting heatmap and relevance from LRP, Deep Taylor and Guided Backprop

In [1]:
pip install tensorflow==1.15

You should consider upgrading via the '/Users/vidhi/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install innvestigate

You should consider upgrading via the '/Users/vidhi/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np 
import tensorflow as tf
import pandas as pd
import glob
import os
from keras.callbacks import History , ModelCheckpoint
import cv2
from cv2 import imread, createCLAHE 
from keras.optimizers import Adam 
#from glob import globfrom keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, UpSampling1D, RepeatVector,TimeDistributed
from keras.layers import Embedding
from keras.models import Model
from keras.layers import Input, GaussianNoise
from keras.models import load_model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.resnet import ResNet101, preprocess_input
from keras.optimizers import SGD, Adam, RMSprop
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.densenet import DenseNet121
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.inception_v3 import  InceptionV3
from keras.utils.vis_utils import plot_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam 
from sklearn.model_selection import train_test_split
import zipfile
from keras import callbacks
from keras import optimizers
from keras import callbacks
from keras.initializers import RandomNormal
from keras.applications import inception_resnet_v2
import innvestigate
import innvestigate.utils as iutils
#from iutils import b
import innvestigate.utils.visualizations as ivis
import numpy as np
from numpy import load
from numpy import save

Using TensorFlow backend.


In [ ]:
#loading the model
model_densenet = load_model('Densenet2-20.hdf5')

In [5]:
#removing the softmax layer from the loaded model
partial_model = Model(
    inputs=model_densenet.inputs,
    outputs=iutils.keras.graph.pre_softmax_tensors(model_densenet.outputs),
    name=model_densenet.name,
)

In [6]:
#preprocess and generate Heatmap for XAI images
def postprocess(X):
    X = X.copy()
    X = iutils.postprocess_images(X)
    return X
def heatmap(X):
    return ivis.heatmap(X)

In [7]:
#analyzing every image and generating heatmaps and relevances
def XAI(Xai_image, method):
  image = Xai_image.reshape(-1,256,256,3)
  analyzer = innvestigate.create_analyzer(method, partial_model)
  relevance  = analyzer.analyze(image)
  result_processed = postprocess(relevance)
  heat_map = heatmap(result_processed)
  return heat_map[0], relevance[0]

In [8]:
#loading testing data
batch_1 = glob.glob('Testing_batch2/*')
path_batch1 = []
for i in range(len(batch_1)):
    s = batch_1[i].split('/')
    path_batch1.append(s[1])

In [9]:
path_batch2 = []
for i in range(len(batch_1)):
    s = path_batch1[i].split('.')
    path_batch2.append(s[0])

In [ ]:
#getting heatmap and relevance for deep taylor 
for im in range(30):
    path1 = 'Testing_batch2/' + path_batch1[im] 
    path1_write = 'Heatmap_deeptaylor/' + path_batch2[im] 
    path2_write = 'Relevances_deeptaylor/' + path_batch2[im] 
    print(path1_write)
    ima = cv2.imread(path1)
    maps,rel = XAI(ima,'deep_taylor') 
    save(path1_write,maps)
    save(path2_write,rel)

In [ ]:
#getting heatmap and relevance for LRP
for im in range(30):
    path1 = 'Testing_batch2/' + path_batch1[im] 
    path1_write = 'Heatmap_LRP/' + path_batch2[im] 
    path2_write = 'Relevances_LRP/' + path_batch2[im] 
    print(path1_write)
    ima = cv2.imread(path1)
    maps,rel = XAI(ima,'lrp.alpha_1_beta_0_IB') 
    save(path1_write,maps)
    save(path2_write,rel)

In [ ]:
#getting heatmaps and relevance for Guided Backprop
for im in range(30):
    path1 = 'Testing_batch2/' + path_batch1[im] 
    path1_write = 'Heatmap_GB/' + path_batch2[im] 
    path2_write = 'Relevances_GB/' + path_batch2[im] 
    print(path1_write)
    ima = cv2.imread(path1)
    maps,rel = XAI(ima,'guided_backprop') 
    save(path1_write,maps)
    save(path2_write,rel)